In [24]:
import re
import csv
import random

In [25]:
# список продуктов
words = ["ряженка", "кефир", "йогурт", "молоко", "творог", "сливки", "сливочное масло", "сметана", "сыр"]

# список брендов
brands = ["Простоквашино", "Домик в деревне", "Веселый молочник", "Savushkin"]

# список процентов (включая слитные формы и без %)
percents = ["3.2 %", "8.5 %", "1 %", "10 %", "2 процентов", "12.5 процентов", "13.5%", "5%", "4", "3.2"]

# список объемов
volumes = []

# имя выходного csv
outfile = "train_new_percent.csv"

In [ ]:
# список продуктов
words = []

# список брендов
brands = []

# список процентов (включая слитные формы и без %)
percents = []

# список объемов
volumes = []

# имя выходного csv
outfile = "train_new_volume.csv"

In [26]:
# регулярки
percent_re = re.compile(r"\d+(\.\d+)?\s*%|\d+(\.\d+)?%|\d+(\.\d+)?\s+процентов|\d+(\.\d+)?")
volume_re = re.compile(
    r"\d+(\.\d+)?\s*(л|литр(а|ов)?|мл|г|грамм(а|ов)?|кг|килограмм(а|ов)?)"
    r"|\d+(\.\d+)?(л|литр(а|ов)?|мл|г|грамм(а|ов)?|кг|килограмм(а|ов)?)"
)

In [27]:
def annotate_multitoken(text, span, label):
    """
    Делает BIO-разметку для многословных сущностей
    span = (start, end)
    """
    result = []
    substring = text[span[0]:span[1]]
    parts = substring.split(" ")
    offset = span[0]
    for i, p in enumerate(parts):
        start = offset
        end = start + len(p)
        if i == 0:
            result.append((start, end, f"B-{label}"))
        else:
            result.append((start, end, f"I-{label}"))
        offset = end + 1  # +1 за пробел
    return result

In [28]:
def annotate_percent(text):
    """Аннотация процентов"""
    annotations = []
    for m in percent_re.finditer(text):
        s, e = m.span()
        token = m.group()

        if token.endswith("%") and not token.endswith(" %"):  # слитная форма
            annotations.append((s, e, "B-PERCENT"))
        elif "%" in token and " " in token:  # число + пробел + %
            num_match = re.match(r"\d+(\.\d+)?", token)
            if num_match:
                num_end = s + len(num_match.group())
                annotations.append((s, num_end, "B-PERCENT"))
                annotations.append((num_end + 1, e, "I-PERCENT"))
        elif "процентов" in token:
            num_match = re.match(r"\d+(\.\d+)?", token)
            if num_match:
                num_end = s + len(num_match.group())
                annotations.append((s, num_end, "B-PERCENT"))
                word_pos = text.find("процентов", s, e)
                if word_pos != -1:
                    annotations.append((word_pos, word_pos + len("процентов"), "I-PERCENT"))
        else:  # просто число
            annotations.append((s, e, "B-PERCENT"))
    return annotations

In [29]:
def annotate_volume(text):
    """Аннотация объёмов и масс с BIO-разметкой"""
    annotations = []
    for m in volume_re.finditer(text):
        s, e = m.span()
        token = m.group()

        # если число и единица слитно, например "200мл" или "1кг"
        if re.match(r"\d+(\.\d+)?(л|литр(а|ов)?|мл|г|грамм(а|ов)?|кг|килограмм(а|ов)?)", token):
            annotations.append((s, e, "B-VOLUME"))

        # если через пробел: "200 мл", "1 литр", "5 килограммов"
        elif re.match(r"\d+(\.\d+)?\s+", token):
            num_match = re.match(r"\d+(\.\d+)?", token)
            if num_match:
                num_end = s + len(num_match.group())
                annotations.append((s, num_end, "B-VOLUME"))
                unit_text = token[num_end - s :].strip()
                unit_start = text.find(unit_text, num_end, e)
                if unit_start != -1:
                    annotations.extend(annotate_multitoken(text, (unit_start, unit_start + len(unit_text)), "VOLUME"))

        else:  # fallback
            annotations.append((s, e, "B-VOLUME"))

    return annotations

In [30]:
def annotate(text, use_volume=False):
    annotations = []

    # TYPE
    for w in words:
        pos = text.find(w)
        if pos != -1:
            annotations.extend(annotate_multitoken(text, (pos, pos + len(w)), "TYPE"))

    # BRAND
    for b in brands:
        pos = text.find(b)
        if pos != -1:
            annotations.extend(annotate_multitoken(text, (pos, pos + len(b)), "BRAND"))

    # PERCENT или VOLUME
    if use_volume:
        annotations.extend(annotate_volume(text))
    else:
        annotations.extend(annotate_percent(text))

    return sorted(annotations, key=lambda x: x[0])

In [31]:
# генерация
with open(outfile, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerow(["sample", "annotation"])

    for _ in range(3):
        for w in words:
            for p in percents:
                if random.random() < 0.5:
                    b = random.choice(brands)
                    text = f"{w} {b} {p}"
                else:
                    text = f"{p} {w}"
                ann = annotate(text, use_volume=False)
                writer.writerow([text, str(ann)])

            for v in volumes:
                if random.random() < 0.5:
                    b = random.choice(brands)
                    text = f"{w} {b} {v}"
                else:
                    text = f"{v} {w}"
                ann = annotate(text, use_volume=True)
                writer.writerow([text, str(ann)])

print(f"✅ Данные с PERCENT и VOLUME сохранены в {outfile}")

✅ Данные с PERCENT и VOLUME сохранены в train_new_percent.csv
